In [ ]:
# Importing (using/include) packages and files needed for the code to run
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics
using Dates
using SpecialFunctions
using Roots
using NLsolve
using HDF5
include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")
include("GridOperations.jl")
# Note: that we import pyplot last to avoid a name conflict with grid
using PyPlot
include("Visualization.jl")
include("Topo.jl")
include("Outputs.jl")

# Stefan Problem - test melting and solidification

Step 1 - import packages

Step 2 - Define a function that relates entropy and temperature/pressure. 
$$
T = f_1(P,S)
$$

$$
T = \exp \left( \frac{S}{C_V} + \ln (T_{ref}) - \frac{\Delta H_{fus} }{C_V T_M}X \right)
$$

and 
$$
S = f_2(P,T)
$$
$$
S = C_V (\ln (T) - \ln(T_{ref})) + \frac{\Delta H_{fus}}{T_m}X
$$

In [ ]:
# functions that relates entropy and temperature/pressure are define in TemperatureEntropy.jl file
include("TemperatureEntropy.jl")


Step 3 - Write functions to set up the initial condition for the Stefan problem

- The box is filled entirely with one material
- The sides are insulating
- Density is constant and equal to 1000 kg/m$^3$
- The surface is at T=100K, the bottom is at T=273 K
- The liquid is isothermal with $T=T_m$.
- The solid has initial $T(z)$ given by the Stefan solution (Turcotte and Schubert section 4.18)
$$
\theta = \frac{erf(\eta)}{erf(\lambda_1)}
$$
- Given the initial temperature, compute initial entropy S on the markers.

In [ ]:
function initial_ice_depth(x::Float64)
    return options["ym"]
end 

struct Materials
    alpha::Vector{Float64} # Thermal expansion (1/K)
    rho0::Vector{Float64} # Density (kg/m^3)
    Hr::Vector{Float64} # Radiogenic heat production (W/m^3)
    Cp::Vector{Float64} # Heat capacity (J/kg*K)
    kThermal::Vector{Float64} # Thermal conductivity (W/m*K)
    function Materials()
        new([0.0],[1e3],[0.0],[2.1e3],[2.2])
    end    
end

function update_marker_prop!(markers::Markers,materials::Materials)
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    X = markers.scalarFields["X"]
    S = markers.scalarFields["S"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        markers.scalars[rho,i] = materials.rho0[mmat[i]]
    end
end

function update_marker_T_X!(markers::Markers,options::Dict)
    T = markers.scalarFields["T"]
    X = markers.scalarFields["X"]
    S = markers.scalarFields["S"]    
    for i in 1:markers.nmark
        markers.scalars[T,i],markers.scalars[X,i] = compute_T_X_from_S((markers.scalars[S,i]),options)
    end
end

function initial_conditions!(markers::Markers,materials::Materials,options::Dict)
    material = markers.integerFields["material"]
    S = markers.scalarFields["S"]
    X = markers.scalarFields["X"]
    T = markers.scalarFields["T"]
    alpha = markers.scalarFields["alpha"]
    Cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    kThermal = markers.scalarFields["kThermal"]
    
    # Setting up Stefan conidtion
    lambda1 = get_lambda1(options)
    t = get_t(lambda1,options)
    
    # Note: the melt fraction is defining our layers or ice and water.
    # water - X = 1.0 
    # ice - X = 0.0
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        hice = initial_ice_depth(mx)
        
        if my > hice # subsurface global ocean
            markers.integers[material,i] = 1
            markers.scalars[alpha,i] = materials.alpha[1]
            markers.scalars[Cp,i] = materials.Cp[1]
            markers.scalars[Hr,i] = materials.Hr[1]
            markers.scalars[kThermal,i] = materials.kThermal[1]
            markers.scalars[T,i] = 273.0
            markers.scalars[X,i] = 1.0
            markers.scalars[S,i] = compute_S_from_T_X(markers.scalars[X,i],markers.scalars[T,i],options)
        elseif my < hice # icy shell
            markers.integers[material,i] = 1
            markers.scalars[alpha,i] = materials.alpha[1]
            markers.scalars[Cp,i] = materials.Cp[1]
            markers.scalars[Hr,i] = materials.Hr[1]
            markers.scalars[kThermal,i] = materials.kThermal[1]
            markers.scalars[T,i] = stefan_initial_condition(get_theta(my,t,lambda1),options)
            markers.scalars[X,i] = 0.0
            markers.scalars[S,i] = compute_S_from_T_X(markers.scalars[X,i],markers.scalars[T,i],options)
        end
    end 
    # end loop over markers
    update_marker_prop!(markers,materials)
end

function get_interface(grid::CartesianGrid,mat::Matrix{Float64},contour_value::Float64)
    # Finding interfaces
    interface_position = zeros(Float64,grid.nx+1);
    for j in 1:grid.nx+1
        i = 1
        while i <= grid.ny
            if mat[i,j] == contour_value
                interface_position[j] = grid.yc[i]
                break
            elseif mat[i+1,j] < contour_value
                # interface is located within this cell.
                interface_position[j] = grid.yc[i] + ((grid.yc[i+1]-grid.yc[i])/(mat[i+1,j]-mat[i,j]))*(contour_value-mat[i,j])
                break
            end
            i = i+1
        end
    end
    return interface_position
end

# Main part of code
- At each timestep, begin by computing T(S,X) on the markers
- Iteratively solve the temperature equation. Start with a guess S_new = S_old
    - For T_new, compute $q_{cond} = \nabla \cdot (k \nabla T_{new})$
    - Solve diffusion equation for S_new
    - Given S_new, update T_new
    - Iterate until converged...
- Given the new entropy solution on the cell centers, update the entropy and melt fraction on the markers.

In [ ]:
function model_run(options::Dict)
    W = 1e4
    H = 2e4
    ny = 50
    nx = 51 
    gx = 0.0
    gy = 0.113
    
    # -1 = insulating, 1 = constant temp 
    Tbctype = [-1,-1,1,1] #left, right, top, bottom
    Tbcval = [0.0,0.0,100.0,273.0] #left, right, top, bottom
    markx = 6
    marky = 6
    seconds_in_year = 3.15e7
    # plot interval should be in seconds
    plot_interval = 1e6*seconds_in_year # 1 Myr 
    
    end_time = 3e7*seconds_in_year
    dtmax = plot_interval
    grid = CartesianGrid(W,H,nx,ny)
    println("Grid resolution(ny x nx) : $ny x $nx")
    println("Cell size in the x-direction is $(grid.W/(grid.nx-1))")
    println("Cell size in the y-direction is $(grid.H/(grid.ny-1))")
    
    materials = Materials()
    println("Creating Markers...")
    @time markers = Markers(grid,["alpha","T","rho","Cp","Hr","kThermal","S","X"],["material"] ; nmx=markx,nmy=marky,random=false)
    println("Initial condition...")
    @time initial_conditions!(markers,materials,options)

    local time_plot = []
    local X_contour = []
    local X_contour_array = []
    
    ### Setting up agruments for termination criteria ###
    max_step::Int64=-1
    max_time::Float64=-1.0
    max_time = max_time == -1.0 ? typemax(Float64) : max_time
    max_step = max_step == -1 ? typemax(Int64) : max_step

    time = 0.0
    iout= 0
    last_plot = 0.0
    dt = dtmax
    
    rho_c = nothing
    kThermal_vx = nothing
    kThermal_vy = nothing
    Hr = nothing
    kThermal = nothing
    dTmax = nothing
    dTemp = nothing
    Tlast = nothing
    Slast = nothing
    Xlast = nothing
    q_vx = nothing
    q_vy = nothing
    melt_fraction_contour = nothing
    Tnew = nothing
    Snew = nothing 
    Xnew = nothing

    # Initial
    ### Transfer properties markers -> nodes ###
    # Cell Centers
    Xlast,Tlast = marker_to_stag(markers,grid,["X","T"],"center")
    # Interpolating entropy using rhoT as weight
    rhoT = markers.scalars[markers.scalarFields["rho"],:] .* markers.scalars[markers.scalarFields["T"],:]
    Slast, = marker_to_stag(markers,grid,["S"],"center",extra_weight=rhoT)  
    # Projecting Slast from the cell centers to the markers
    cell_center_to_markers!(markers,grid,Slast,"S")
    # Updating Temperature and Melt fraction on the markers
    update_marker_T_X!(markers,options)

    ### Initial Plots ###
    get_plots(grid,Slast,Tlast,Xlast,"initial")
    
    itime = 1 
    output_dir = "output_subgrid_diff"
    terminate = false
    while !terminate

        ### update the markers properties ###
        update_marker_prop!(markers,materials)
        ### Transfer properties markers -> nodes ###
        # Cell Centers
        Xlast_new,Tlast_new = marker_to_stag(markers,grid,["X","T"],"center")
        rho_c_new,Hr_new,kThermal_new = marker_to_stag(markers,grid,["rho","Hr","kThermal"],"center")
        # Interpolating entropy using rhoT as weight
        rhoT = markers.scalars[markers.scalarFields["rho"],:] .* markers.scalars[markers.scalarFields["T"],:]
        Slast_new, = marker_to_stag(markers,grid,["S"],"center",extra_weight=rhoT)       
        # Vx and Vy nodes:
        kThermal_vx_new, = marker_to_stag(markers,grid,["kThermal",],"vx")
        kThermal_vy_new, = marker_to_stag(markers,grid,["kThermal",],"vy")
        
        # Copy field data
        rho_c = copy(rho_c_new)
        kThermal_vx = copy(kThermal_vx_new)
        kThermal_vy = copy(kThermal_vy_new)
        Hr = copy(Hr_new)
        kThermal = copy(kThermal_new)
        Tlast = copy(Tlast_new)    
        Slast = copy(Slast_new)
        Xlast = copy(Xlast_new)

        diffusion_timestep = calculate_diffusion_timestep(grid,options)
        if itime > 1
            this_dtmax = min(1.2*dt,dtmax)
        else
            this_dtmax = dtmax
        end
        if this_dtmax > diffusion_timestep
            dt = diffusion_timestep
        else 
            dt = this_dtmax
        end

        last_T_norm = NaN
        T_norm = NaN
        dT = nothing
        dTmax = Inf
        dS = nothing
        dSmax = Inf
        tolerance = 1e-8
        dTnorm = []
        ititer = []
        titer = 1 
        max_titer = 300
        for titer=1:max_titer
            
            # Computing conductive heat flux (using Tlast yields an explicit scheme)
            q_vx,q_vy = compute_q_cond(grid,Tlast,kThermal_vx,kThermal_vy) # W/m^2
            
            # Computing the new entropy (using Tlast yields an explicit scheme)
            Snew = compute_S_new(grid,Tlast,rho_c,Hr,q_vx,q_vy,Slast,dt);
    
            # Updating the new temperature and new melt fraction from the new entropy
            Tnew,Xnew = update_T_X_from_S(Snew,options)             
            Tnew,Xnew,Snew = ghost_nodes_center_TXS(grid,Tnew,Xnew,Snew,Tbctype,Tbcval,options)

            # Computing residual for entropy
            residual_S = compute_entropy_residual(grid,Tlast,rho_c,Hr,q_vx,q_vy,Slast,Snew,dt)
            Snorm = norm(residual_S[2:ny,2:nx])
       
            if titer > 1
                last_T_norm = T_norm;
                T_norm = norm(Tnew[2:grid.ny,2:grid.nx]);
                push!(dTnorm,abs(T_norm-last_T_norm))
                push!(ititer,titer)
            else
                T_norm = norm(Tnew[2:end-1,2:end-1]);
                push!(dTnorm,abs(T_norm-last_T_norm))
                push!(ititer,titer)
            end
            
            # Computing the maximum temperature change
            dT = Tnew - Tlast
            dTmax = maximum(abs.(dT[2:grid.ny,2:grid.nx]))

            # Computing the maximum entropy change
            dS = Snew - Slast
            dSmax = maximum(abs.(dS[2:grid.ny,2:grid.nx]))

            # Checking for convergence:
            if Snorm < tolerance
                break
            elseif titer == max_titer
                terminate = true
                @error("Did not converged")
            iout += 1
            elseif any(isnan.(dT))
                terminate = true
                @error("NaN or Inf apperred")
            end
        end

        # Updating entropy on the markers by projecting dS from the cell centers to the markers
        # cell_center_change_to_markers!(markers,grid,dS,"S")
        # update_marker_T_X!(markers,options)
        dS_subgrid_node = subgirdSdiff!(grid,markers,Slast,dt,options,diff_coeff=1.0)
        dS_remaining = dS-dS_subgrid_node
        cell_center_change_to_markers!(markers,grid,dS_remaining,"S")
        update_marker_T_X!(markers,options)

        Slast_new = copy(Snew)
        Xlast_new = copy(Xnew)
        Tlast_new = copy(Tnew)
        
        melt_fraction_contour = get_interface(grid,-Xnew,-0.5)
        append!(X_contour_array,[melt_fraction_contour])
        avg_melt_fraction_contour = mean(melt_fraction_contour)
        append!(X_contour,avg_melt_fraction_contour)
        append!(time_plot,time)  

        # Checking Termination Criteria, time is in Myr, amplitude is in meters
        if time >= max_time || itime >= max_step || (X_contour[itime] - X_contour[1]) > options["thickening criteria"]
            terminate = true
            ### Final Plots ###
            get_plots(grid,Snew,Tnew,Xnew,"final")
        end

        if time == 0.0 || mod(itime,10) == 0 || terminate
            last_plot = time 
            # Markers output
            name1 = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
            visualization(markers,time/seconds_in_year;filename=name1);
            iout += 1
        end

        time += dt
        if mod(itime,200) == 0
            println("Ice shell as thicken by ",(X_contour[itime] - X_contour[1])/1e3,"km")
            println("time = ",time/seconds_in_year," yr, ",time/seconds_in_year/1e3," Kyr, ",time/seconds_in_year/1e6," Myr") 
        end
        itime += 1
    end
    return grid,time_plot,X_contour_array,X_contour,itime,Tnew
end

In [ ]:
options = Dict()
options["latent heat of fusion"] = 3.34e5 #J/kg
options["specific heat of ice"] = 2.1e3 # J/kg*K (ice)
options["density of ice"] = 1e3 # kg/m^3
options["thermal conductivity of ice"] = 2.2 # W/m*K
options["thermal diffusivity"] = options["thermal conductivity of ice"] / (options["density of ice"]*options["specific heat of ice"]) # m^2/s
options["Tm"] = 273.0 # K
options["To"] = 100.0 # K
options["ym"] = 1e4 # m 
# Define thickening criteria (this will stop the model when the ice has thicken by the value set by thickening criteria)
options["thickening criteria"] = 2e3 # m
grid,time_plot,X_contour_array,X_contour,itime,Tnew = model_run(options);
compute_stefan_temp_solution(grid,Tnew,X_contour,time_plot,itime)
thickness_over_time(grid,X_contour_array,time_plot,itime)